# Import packages

# Auxiliary Functions

In [1]:
import numpy as np
from copy import deepcopy
from scipy.io import loadmat
import os
import time

In [2]:
def ldet(A):
    sign, value = np.linalg.slogdet(A)
    if sign > 0:
        return value
    else:
        return -np.inf
def ldet_objval(A,x):
    return ldet(np.dot(np.dot(A.T, np.diag(x.T[0])), A))

In [3]:
def init_binary(A,R,s,m,n):
    U, S, VH = np.linalg.svd(A, full_matrices=True)
    x = np.zeros((n,1))
    for j in range(n):
        for i in range(s):
            x[j] += (U[j,i]**2)
    x_save = deepcopy(x)
    x = np.zeros((n,1))
    for row in R:
        x[row-1] = 1
        x_save[row-1] = 0

    for i in range(s-m):
        max_indice = np.argmax(x_save)
        x[max_indice] = 1
        x_save[max_indice] = 0
    zlb = ldet_objval(A, x)
    xlb = x
    return xlb, zlb

def init_greedy(A,R,s,m,n):
    U, S, VH = np.linalg.svd(A, full_matrices=True)
    x = np.zeros((n,1))
    k = min(s,m)
    for j in range(n):
        for i in range(k):
            x[j] += (S[i] * U[j,i]**2)
    x_save = deepcopy(x)
    x = np.zeros((n,1))
    for row in R:
        x[row-1] = 1
        x_save[row-1] = 0

    for i in range(s-m):
        max_indice = np.argmax(x_save)
        x[max_indice] = 1
        x_save[max_indice] = 0

    zlb = ldet_objval(A, x)
    xlb = x
    return xlb, zlb

In [4]:
def LSFI(A,n,x_init,z_lb): # Local Search First Improvement
    x = deepcopy(x_init)
    flag = True
    while flag:
        flag = False
        for i in range(n):
            if x[i] > 0:
                x[i] = 0
                for j in range(n):
                    if j != i and x[j] == 0:
                        x[j] = 1
                        z_lb_new = ldet_objval(A, x)
                        if z_lb_new > z_lb:
                            z_lb = z_lb_new
                            flag = True
                            break
                        else:
                            x[j] = 0
                if flag:
                    break
                else:
                    x[i] = 1
    return x, z_lb

def LSFP(A,n,x_init,z_lb): # Local Search First Improvement Plus
    x = deepcopy(x_init)
    flag = True
    leave_x, enter_x = 0, 0
    while flag:
        flag = False
        for i in range(n):
            if x[i] > 0:
                x[i] = 0
                for j in range(n):
                    if j != i and x[j] == 0:
                        x[j] = 1
                        z_lb_new = ldet_objval(A, x)
                        if z_lb_new > z_lb:
                            leave_x, enter_x = i, j
                            z_lb = z_lb_new
                            flag = True
                        x[j] = 0
                if flag:
                    break
                else:
                    x[i] = 1
        if flag:
            # x[leave_x] = 0
            x[enter_x] = 1
    return x, z_lb

def LSBI(A,n,x_init,z_lb): # Local Search Best Improvement
    x = deepcopy(x_init)
    flag = True
    leave_x, enter_x = 0, 0
    while flag:
        flag = False
        for i in range(n):
            if x[i] > 0:
                x[i] = 0
                for j in range(n):
                    if j != i and x[j] == 0:
                        x[j] = 1
                        z_lb_new = ldet_objval(A, x)
                        if z_lb_new > z_lb:
                            leave_x, enter_x = i, j
                            z_lb = z_lb_new
                            flag = True
                        x[j] = 0
                x[i] = 1
        if flag:
            x[leave_x] = 0
            x[enter_x] = 1
    return x, z_lb


In [5]:
def run_local_search(A, R, n, m, s):
    x_init_bin, z_init_bin = init_binary(A, R, s, m, n)
    x_init_gre, z_init_gre = init_greedy(A, R, s, m, n)
    X = [x_init_bin, x_init_gre]
    Z = [z_init_bin, z_init_gre]

    X_init = [x_init_bin, x_init_gre]
    for x_init in X_init:
        x, z = LSFI(A, n, x_init, z_init_bin)
        X.append(x)
        Z.append(z)
        x, z = LSFP(A, n, x_init, z_init_bin)
        X.append(x)
        Z.append(z)
        x, z = LSBI(A, n, x_init, z_init_bin)
        X.append(x)
        Z.append(z)
    z_heur = np.max(Z)
    indsX = np.where(Z == z_heur)[0]
    x_heur = X[indsX[0]]

    sum_x = []
    max_x = []
    for x in X:
        sum_x.append(np.sum(x))
        max_x.append(np.max(x))
    return x_heur, z_heur, (X, Z, sum_x, max_x, indsX)    

In [6]:
instances = [f"Instance_{n}_{i}" for n in [40,60,80,100,140,180,200,240,280,300] for i in range(1,4)]
print(instances)

['Instance_40_1', 'Instance_40_2', 'Instance_40_3', 'Instance_60_1', 'Instance_60_2', 'Instance_60_3', 'Instance_80_1', 'Instance_80_2', 'Instance_80_3', 'Instance_100_1', 'Instance_100_2', 'Instance_100_3', 'Instance_140_1', 'Instance_140_2', 'Instance_140_3', 'Instance_180_1', 'Instance_180_2', 'Instance_180_3', 'Instance_200_1', 'Instance_200_2', 'Instance_200_3', 'Instance_240_1', 'Instance_240_2', 'Instance_240_3', 'Instance_280_1', 'Instance_280_2', 'Instance_280_3', 'Instance_300_1', 'Instance_300_2', 'Instance_300_3']


In [7]:
for instance_name in instances:
        instance = loadmat(os.path.join('../instances', instance_name))
        A = instance["A"]
        n = A.shape[0]
        m = A.shape[1]
        s = int(n/2)
        R = instance['R']
        time_ini = time.time()
        x_ls, z_ls, info_ls = run_local_search(A, R, n, m, s)
        time_end = time.time()
        print(f"Finished LS {instance_name} - Result: {z_ls} - Time: {time_end - time_ini}")

Finished LS Instance_40_1 - Result: -1.5538065375157495 - Time: 0.23537969589233398
Finished LS Instance_40_2 - Result: -0.7013141749532328 - Time: 0.40554380416870117
Finished LS Instance_40_3 - Result: -1.417194747674704 - Time: 0.3327031135559082
Finished LS Instance_60_1 - Result: -1.7811835092404427 - Time: 1.4317052364349365
Finished LS Instance_60_2 - Result: -2.3348248346041665 - Time: 1.5300977230072021
Finished LS Instance_60_3 - Result: -1.40011221194216 - Time: 0.7731006145477295
Finished LS Instance_80_1 - Result: -2.7777165416793443 - Time: 4.012126684188843
Finished LS Instance_80_2 - Result: -2.426260131985708 - Time: 2.5497827529907227
Finished LS Instance_80_3 - Result: -2.4546132059018886 - Time: 4.655780792236328
Finished LS Instance_100_1 - Result: -2.502724115620391 - Time: 3.981982946395874
Finished LS Instance_100_2 - Result: -2.874370133356307 - Time: 5.445460081100464
Finished LS Instance_100_3 - Result: -2.6437356608557825 - Time: 8.655131340026855
Finished L